In [2]:
import struct
import json 
from abc import ABC

In [ ]:
1. 0x0C Spawn Player
2. 0x12 Entity Velocity
3. 0x14 Entity
4. 0x15 Entity Relative Move
5. 0x16 Entity Look
6. 0x17 Entity Look And Relative Move
7. 0x18 Entity Teleport
8. 0x19 Entity Head Look
9. 0x1A Entity Status
10. 0x1C Entity Metadata
11. 0x32 Confirm Transaction

"""
"bool": '?',
"i8": 'b',
"u8": 'B',
"i16": 'h'
"u16": 'H',
"i32": 'i',
"i64": 'I'
"f32": 'f',
"f64": 'd',
"UUID" : 'H',
"string": ['varint', 's'],
"varint": 
"entityMetadata"
"slot"
"position": "varint"
"restBuffer"
"optionalNbt"
"""

In [4]:
class Packet(ABC):
    def __init__(self, timestamp: int, length: int, bytearray, id=None, type=None):
        self.timestamp = timestamp
        self.length = length
        self.bytearray = bytearray
        self.id = id
        self.type = type
    def __repr__(self) -> str:
        return f'Packet of type: {self.type} with id {self.id} starting at {self.timestamp} with a bytearray {self.bytearray} of length {self.length}'
    def decode(self):
        pass

In [5]:
class SpawnPlayerPacket(Packet):
    def __init__(self, timestamp: int, length: int, bytearray, id:int, type:str):
        super().__init__()

In [10]:
packets = []
#open byte file
#read in packet data
#structure: timestamp of packet (integer) length of packet bytearray (integer) packet(bytearray)
with open("./data/2021_05_02_22_04_36_1/recording.tmcpr", "rb") as f:
    while timestamp := f.read(4):
        timestamp = struct.unpack('>i', timestamp)[0]
        length = struct.unpack('>i', f.read(4))[0]
        bytearray = f.read(length)
        packets.append(
            Packet(
                timestamp=timestamp,
                length=length,
                bytearray=bytearray
            )
        )

In [12]:
packets[0]

Packet of type: None with id None starting at 0 with a bytearray b'\x02$d41d8cd9-8f00-3204-a980-0998ecf8427e\x06Player' of length 45

In [16]:
#skip first packet because its a login success packet
packets = packets[1:]

In [17]:
#read the protocol for 1.8
#Protocol 47 
#JSON from https://github.com/PrismarineJS/minecraft-data/blob/master/data/pc/1.8/protocol.json
protocol = json.load(open('./data/protocol.json', 'r'))

In [19]:
#json dict of packet name and the make up of the packet
packet_json = protocol['play']['toClient']['types']

In [20]:
#creates a mapping between id numbers and names of packets

#gets the mappings from the protocol
packet_types = packet_json['packet'][1][0]['type'][1]['mappings']

#adds packet_ in front of it to work with the keys of packet_json
packet_types_values = ["packet_"+x for x in list(packet_types.values())]

#converts the hex string ids to numbers
packet_types_keys = [int(x, 16) for x in list(packet_types.keys())]

#creates a dictionary of them
packet_types = dict(zip(packet_types_keys, packet_types_values))

In [21]:

#function to read specific type of integer from minecraft
#variable length integer 
def read_var_int(b):
    value = 0
    length = 0
    while True:
        currentByte = b[length]
        value |= (currentByte & 0x7F) << (length * 7)
        length += 1
        if ((value & 0x80) != 0x80):
            break
    return value

In [22]:
#classifies the packet and removes the id byte and modifies length accordingly
for i, packet in enumerate(packets):
    packet.type = packet_types[packet.bytearray[0]]
    packet.id = packet.bytearray[0]
    packet.bytearray = packet.bytearray[1:]
    packet.length-=1